In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
# tsv_file='TH_data_challenge.tsv'
# csv_table=pd.read_table(tsv_file,sep='\t')
# csv_table.to_csv('data_.csv',index=False)

ImportError: No module named matplotlib.pyplot

In [238]:
df = pd.read_csv('data_.csv', sep=',')

In [239]:
Y=df.loc[:, :'dim_is_requested']
df.drop(df.columns[[0,1,2,3,4]], axis=1, inplace=True)
#print df.head()

In [240]:
def myfillna(series):
    if series.dtype is pd.np.dtype(float):
        #print ("found")
        return series.fillna(-1)
    elif series.dtype is pd.np.dtype(object):
        #print ("case2")
        return series.fillna('.')
    else:
        return series

In [241]:
df = df.apply(myfillna)
#print df.head()
print df.dtypes

m_effective_daily_price                               float64
m_pricing_cleaning_fee                                float64
dim_market                                             object
dim_lat                                               float64
dim_lng                                               float64
dim_room_type                                          object
dim_person_capacity                                     int64
dim_is_instant_bookable                                  bool
m_checkouts                                           float64
m_reviews                                             float64
days_since_last_booking                               float64
cancel_policy                                           int64
image_quality_score                                   float64
m_total_overall_rating                                float64
m_professional_pictures                               float64
dim_has_wireless_internet                               int64
ds_night

In [242]:
X = df.select_dtypes(include=[object,bool])
X_other= df.select_dtypes(include=[float,int])
print X.head(3)
print X.shape
print X_other.shape

    dim_market dim_room_type  dim_is_instant_bookable
0  Los Angeles  Private room                    False
1  Los Angeles  Private room                     True
2  Los Angeles  Private room                    False
(184279, 3)
(184279, 38)


In [243]:
from sklearn import preprocessing
print X.columns
# TODO: create a LabelEncoder object and fit it to each feature in X
# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = X.apply(le.fit_transform)
X_2.head(10)

Index([u'dim_market', u'dim_room_type', u'dim_is_instant_bookable'], dtype='object')


,dim_market,dim_room_type,dim_is_instant_bookable
0,0,1,0
1,0,1,1
2,0,1,0
3,0,0,0
4,0,0,0
5,0,1,0
6,0,1,0
7,0,0,0
8,0,2,0
9,0,0,0


In [244]:
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()
# 2. FIT
enc.fit(X_2)
# 3. Transform
onehotlabels = enc.transform(X_2).toarray()
print onehotlabels.shape
print onehotlabels

(184279, 8)
[[1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]]


In [245]:
X_=np.concatenate((onehotlabels, X_other), axis=1)
print X_.shape

(184279, 46)


In [246]:
from sklearn.preprocessing import LabelEncoder

#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object','bool']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [247]:
Y=dummyEncode(Y)
Y=Y.as_matrix()
print Y[0]

[0]


In [249]:
# def indices_to_one_hot(data, nb_classes):
#     """Convert an iterable of indices to one-hot encoded labels."""
#     targets = np.array(data).reshape(-1)
#     return np.eye(nb_classes)[targets]

# Y=indices_to_one_hot(2, Y)
# print Y
print X_[0]
print Y[0]

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  1.10000000e+02  6.00000000e+01  3.40539320e+01 -1.18362970e+02
  2.00000000e+00  2.40000000e+01  1.90000000e+01  3.20000000e+02
  4.00000000e+00  8.30959000e-01  7.40000000e+01  1.60000000e+01
  1.00000000e+00  2.00000000e+00  6.00000000e+00  0.00000000e+00
  7.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  5.00000000e+00  9.00000000e+01  9.20000000e+01 -1.00000000e+00
 -1.00000000e+00  1.33333333e-01  7.88142857e+02  2.41428571e+02
  4.14285714e+01  1.00000000e+00  9.84000000e+03  1.31428576e+00
  1.79591837e+00  5.70000000e+01  4.80000000e+01  7.90000000e+01
  7.50000000e+01  9.80000000e+01]
[0]


In [250]:
from sklearn.model_selection import train_test_split
import warnings
import operator
warnings.filterwarnings("ignore")
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.33, random_state=42)

In [251]:
from sklearn.ensemble import GradientBoostingClassifier
#import xgboost as xgb
import warnings
import heapq
import time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [252]:
clf = RandomForestClassifier(n_estimators=100,bootstrap=True,min_samples_leaf=10,min_samples_split=2,criterion='gini',max_features=3, max_depth=None)
clf=clf.fit(X_train,y_train)


In [253]:
print(clf.feature_importances_)
y_pred=clf.predict(X_test)

[0.003507   0.00730831 0.00173547 0.00260083 0.00243546 0.00067006
 0.00456751 0.00441398 0.08103341 0.01683962 0.02603545 0.02705266
 0.01107026 0.03585913 0.02986899 0.05383178 0.00588166 0.01816496
 0.03579792 0.00708368 0.00090896 0.00680817 0.02404841 0.0120601
 0.00770472 0.01243136 0.02664622 0.094765   0.01073876 0.0102229
 0.04030541 0.01025426 0.00914779 0.10531009 0.02242525 0.02746329
 0.02503862 0.0028502  0.03291179 0.01928094 0.02218826 0.01707427
 0.01719938 0.02142087 0.02221304 0.02282378]


In [254]:
from sklearn.metrics import accuracy_score
print accuracy_score(y_test, y_pred)

0.8156808577113446


In [255]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [256]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=46, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# larger model
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=46, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X_, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))